In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.autograd import Function
import numpy as np
import os
from torchvision import datasets as datasets
from torchvision import transforms as transforms
from torch.utils.data import DataLoader as DataLoader
from math import ceil, floor

from tools import *
from encoding import *
from models import *
from bnn import *

In [2]:
mnist = BNN(name="MNIST", output_dir="mnist_logs/")
mnist.train()

Start/Continue training from epoch 10
Finish training for 10 epochs


In [3]:
verify_net(mnist.net)

#manual_acc(mnist.net, mnist.train_loader)

In [4]:
decomp = decompose_net(mnist.net)
linear, bn = decomp['linear'], decomp['bn']

D = lower_bound(mnist.net)

In [5]:
def r_value(row, m, C):
    return int((row[:m] == 1).sum().item() >= C)

def make_bool(mat):
    mat = mat.cpu().detach().numpy()
    
    return ((mat + 1)/2).astype(int)

In [6]:
from functools import *
from collections import defaultdict

In [7]:
fmt_x = "x%d,%d,%d" # block, row, col
fmt_r = "r%d,%d,%d,%d" # block, row, col (m), C value
variables = defaultdict(str)
var_idx = 1

for block in range(len(linear)-1):
    mat = make_bool(linear[block]['weight'])

    for row in range(mat.shape[0]):
        for col in range(mat.shape[1]):
            variables[fmt_x % (block, row, col)] = var_idx
            var_idx += 1
            
#             for C in range(1, D[block][row] + 1):
#                 variables[fmt_r % (block, row, col, C)] = var_idx
#                 var_idx += 1

In [8]:
len(variables)

6146560

In [9]:
def convert_to_dimacs(cnf_str, variables):
    for word in cnf_str.replace("-", "").split():
        if word in variables:
            cnf_str = cnf_str.replace(word, str(variables[word]))
    
    return cnf_str

In [10]:
def constraint1(mat, block):
    mat = make_bool(mat)
    res = ''
    
    for row in range(mat.shape[0]):
        if r_value(mat[row,:], 0, 1):
            res += (fmt_x % (block, row, 0) + ' ')
        else:
            res += ('-' + fmt_x % (block, row, 0) + ' ')
            
    res = res.strip().split()
    res = list(map(lambda x : x.strip() + ' 0', res))
    
    return '\n'.join(res)

In [11]:
def constraint2(mat, block):
    mat = make_bool(mat)
    
    all_conj = []
    
    for row in range(mat.shape[0]):
        row_conj = []
            
        for C in range(2, D[block][row] + 1):
            row_conj.append(int(not r_value(mat[row,:], 1, C)))
        
        all_conj.append(reduce(lambda x, y : x & y, row_conj))
    
    return all(all_conj)

In [ ]:
def constraint3(mat, block):
    

In [14]:
one = constraint1(linear[2]['weight'], 2)
print(convert_to_dimacs(one, variables))

-3687937 0
-3689505 0
-3691073 0
-3692641 0
-3694209 0
-3695777 0
-3697345 0
-3698913 0
-3700481 0
-3702049 0
-3703617 0
-3705185 0
-3706753 0
-3708321 0
-3709889 0
-3711457 0
-3713025 0
-3714593 0
-3716161 0
-3717729 0
-3719297 0
-3720865 0
-3722433 0
-3724001 0
-3725569 0
-3727137 0
-3728705 0
-3730273 0
-3731841 0
-3733409 0
-3734977 0
-3736545 0
-3738113 0
-3739681 0
-3741249 0
-3742817 0
-3744385 0
-3745953 0
-3747521 0
-3749089 0
-3750657 0
-3752225 0
-3753793 0
-3755361 0
-3756929 0
-3758497 0
-3760065 0
-3761633 0
-3763201 0
-3764769 0
-3766337 0
-3767905 0
-3769473 0
-3771041 0
-3772609 0
-3774177 0
-3775745 0
-3777313 0
-3778881 0
-3780449 0
-3782017 0
-3783585 0
-3785153 0
-3786721 0
-3788289 0
-3789857 0
-3791425 0
-3792993 0
-3794561 0
-3796129 0
-3797697 0
-3799265 0
-3800833 0
-3802401 0
-3803969 0
-3805537 0
-3807105 0
-3808673 0
-3810241 0
-3811809 0
-3813377 0
-3814945 0
-3816513 0
-3818081 0
-3819649 0
-3821217 0
-3822785 0
-3824353 0
-3825921 0
-3827489 0
-3829057 0

In [13]:
assert(constraint2(linear[0]['weight'], 0))
assert(constraint2(linear[1]['weight'], 1))
assert(constraint2(linear[2]['weight'], 2))